In [1]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns

### Loading Dataset:

In [2]:
data = pd.read_csv(r".\data\Social_Network_Ads.csv")
data

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
data.describe()

,User ID,Age,EstimatedSalary,Purchased
count,4.000000e+02,400.000000,400.000000,400.000000
mean,1.569154e+07,37.655000,69742.500000,0.357500
std,7.165832e+04,10.482877,34096.960282,0.479864
min,1.556669e+07,18.000000,15000.000000,0.000000
25%,1.562676e+07,29.750000,43000.000000,0.000000
50%,1.569434e+07,37.000000,70000.000000,0.000000
75%,1.575036e+07,46.000000,88000.000000,1.000000
max,1.581524e+07,60.000000,150000.000000,1.000000


## Feature Engineering

####  1-Convert the categorical variables into numerical form

In [7]:
Sex  = pd.get_dummies(data['Gender'] , drop_first = True)
Sex

,Male
0,1
1,1
2,0
3,0
4,1
...,...
395,0
396,1
397,0
398,1


In [8]:
data['Sex'] = Sex
data = data.drop(['Gender', 'User ID' ], axis =1)

In [9]:
data

,Age,EstimatedSalary,Purchased,Sex
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


#### 2. Standardize the data

In [15]:
data

,Age,EstimatedSalary,Purchased,Sex
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


## Models

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [10]:
x = data[["Age","Sex","EstimatedSalary"]]
y = data['Purchased']

npX = np.array(x).copy()
npy = np.array(y).copy()

X_train  ,X_test , y_train , y_test = train_test_split( x , y , test_size = 0.3 , random_state = 50)

In [11]:
clf_rf = RandomForestClassifier()
clf_dt = DecisionTreeClassifier()
clf_knn = KNeighborsClassifier()
clf_nb = GaussianNB()
clf_lr = LogisticRegression()
clf_svm = SVC() #RBF

In [19]:
Classifiers = ['RandomForest','DecisionTree','KNN','Naive Bayes','LogisticRegression','SVM']
scores = []
models = [clf_rf, clf_dt, clf_knn, clf_nb, clf_lr,clf_svm]
for model in models:
    score = cross_val_score(model, npX, npy, scoring = 'accuracy', cv = 10, n_jobs = -1).mean()
    scores.append(score)

In [20]:
mode = pd.DataFrame(scores, index = Classifiers, columns = ['score']).sort_values(by = 'score',
             ascending = False)
mode

,score
RandomForest,0.8750
Naive Bayes,0.8750
DecisionTree,0.8350
KNN,0.7850
SVM,0.7675
LogisticRegression,0.6425


In [13]:
import pickle
with open(r'.\model\model.pkl','wb') as f:
    pickle.dump(clf_knn,f)

In [30]:
values =[]
age= int(input("Enter your age: "))
EstimatedSalary = int(input("Enter your Estimate Salary: "))
sex= int(input("Enter the gender[male =1 and female =0]: "))
values.extend([age,sex,EstimatedSalary])

Enter your age: 51
Enter your Estimate Salary: 23000
Enter the gender[male =1 and female =0]: 1


In [14]:
fit_model = clf_knn.fit(X_train,y_train)
predicted_val = fit_model.predict([[55,1,40000]])
predicted_val[0]

c:\Users\RENS\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\RENS\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


1

In [15]:
with open(r'.\model\model.pkl', 'rb') as f:
    clf2 = pickle.load(f)

In [16]:
# values = []
age,sex,EstimatedSalary = 23, 1 , 950
values = [[age,sex,EstimatedSalary]]
clf2.fit(X_train,y_train)
clf2.predict(values)

c:\Users\RENS\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\RENS\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array([0], dtype=int64)